In [1]:
import json
import requests

In [2]:
def pretty_print(data: dict):
    print(json.dumps(data, indent=4))

In [3]:
api_key = 'HLWXBVGZWBOSWKXH'

def retrieve_data(function: str, symbol: str, api_key: str) -> dict:
    """
    Retrieves data from AlphaVantage's open API.
    Documentation located at: https://www.alphavantage.co/documentation
    """
    # query from API
    url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    # read output
    data = response.text
    # parse output
    parsed = json.loads(data)

    return parsed

In [4]:
def get_latest_balance_sheet(symbol: str, api_key: str) -> dict:
    """
    Retrieves latest balance sheet of company specified in symbol.
    """
    balance_sheets = retrieve_data('BALANCE_SHEET', symbol, api_key)['annualReports']
    latest_balance_sheet = balance_sheets[0]
    return latest_balance_sheet

def get_second_latest_balance_sheet(symbol: str, api_key: str) -> dict:
    """
    Retrieves second latest balance sheet of company specified in symbol.
    """
    balance_sheets = retrieve_data('BALANCE_SHEET', symbol, api_key)['annualReports']
    second_latest_balance_sheet = balance_sheets[1]
    return second_latest_balance_sheet

def get_latest_income_statement(symbol: str, api_key: str) -> dict:
    """
    Retrieves latest income statement of company specified in symbol.
    """
    income_statements = retrieve_data('INCOME_STATEMENT', symbol, api_key)['annualReports']
    latest_income_statement = income_statements[0]
    return latest_income_statement

In [5]:
class FinancialStatementAnalyzer:
    def __init__(self, balance_sheet: dict, prior_balance_sheet: dict, income_statement: dict):
        self.balance_sheet = balance_sheet
        self.prior_balance_sheet = prior_balance_sheet
        self.income_statement = income_statement
        
    def net_profit_margin(self):
        net_income = float(self.income_statement['netIncome'])
        revenue = float(self.income_statement['totalRevenue'])
        return round(net_income/revenue, 4)
    
    def asset_turnover(self):
        revenue = float(self.income_statement['totalRevenue'])
        average_total_assets = (float(self.balance_sheet['totalAssets']) +
                               float(self.prior_balance_sheet['totalAssets']))/2
        return round(revenue/average_total_assets, 4)
    
    def equity_multiplier(self):
        average_total_assets = (float(self.balance_sheet['totalAssets']) +
                               float(self.prior_balance_sheet['totalAssets']))/2
        average_shareholders_equity = (float(self.balance_sheet['totalShareholderEquity']) +
                                      float(self.prior_balance_sheet['totalShareholderEquity']))/2
        return round(average_total_assets/average_shareholders_equity, 3)
    
    def roe(self):
        net_income = float(self.income_statement['netIncome'])
        average_shareholders_equity = (float(self.balance_sheet['totalShareholderEquity']) +
                                      float(self.prior_balance_sheet['totalShareholderEquity']))/2
        return round(net_income/average_shareholders_equity, 4)



In [7]:
import time

stocks = ['TSLA', 'F', 'GM']

# simple DuPont analysis
print('DuPont Analysis')
for stock in stocks:
    latest_bs = get_latest_balance_sheet(stock, api_key)
    prior_bs = get_second_latest_balance_sheet(stock, api_key)
    latest_is = get_latest_income_statement(stock, api_key)
    analyzer = FinancialStatementAnalyzer(latest_bs, prior_bs, latest_is)

    # print ROE and DuPont analysis ratios
    print(f'{stock}\tProfit Margin: {analyzer.net_profit_margin()}\tAsset Turnover: {analyzer.asset_turnover()}\t\
    Equity Multiplier: {analyzer.equity_multiplier()}\tROE: {analyzer.roe()}')
    time.sleep(60)

DuPont Analysis
TSLA	Profit Margin: 0.1541	Asset Turnover: 1.1277	    Equity Multiplier: 1.929	ROE: 0.3353
F	Profit Margin: -0.0125	Asset Turnover: 0.6163	    Equity Multiplier: 5.59	ROE: -0.0432
GM	Profit Margin: 0.0634	Asset Turnover: 0.6162	    Equity Multiplier: 3.989	ROE: 0.1558


In [47]:
pretty_print(retrieve_data('OVERVIEW', 'NVDA', api_key))

{
    "Symbol": "NVDA",
    "AssetType": "Common Stock",
    "Name": "NVIDIA Corporation",
    "Description": "Nvidia Corporation is an American multinational technology company incorporated in Delaware and based in Santa Clara, California. It designs graphics processing units (GPUs) for the gaming and professional markets, as well as system on a chip units (SoCs) for the mobile computing and automotive market.",
    "CIK": "1045810",
    "Exchange": "NASDAQ",
    "Currency": "USD",
    "Country": "USA",
    "Sector": "MANUFACTURING",
    "Industry": "SEMICONDUCTORS & RELATED DEVICES",
    "Address": "2701 SAN TOMAS EXPRESSWAY, SANTA CLARA, CA, US",
    "FiscalYearEnd": "January",
    "LatestQuarter": "2023-04-30",
    "MarketCapitalization": "1147586740000",
    "EBITDA": "6090000000",
    "PERatio": "243.25",
    "PEGRatio": "4.041",
    "BookValue": "9.91",
    "DividendPerShare": "0.16",
    "DividendYield": "0.0004",
    "EPS": "1.91",
    "RevenuePerShareTTM": "10.44",
    "Profi

In [48]:
pretty_print(retrieve_data('CASH_FLOW', 'NVDA', api_key))

{
    "symbol": "NVDA",
    "annualReports": [
        {
            "fiscalDateEnding": "2023-01-29",
            "reportedCurrency": "USD",
            "operatingCashflow": "5641000000",
            "paymentsForOperatingActivities": "438000000",
            "proceedsFromOperatingActivities": "None",
            "changeInOperatingLiabilities": "-299000000",
            "changeInOperatingAssets": "3249000000",
            "depreciationDepletionAndAmortization": "1544000000",
            "capitalExpenditures": "1833000000",
            "changeInReceivables": "-822000000",
            "changeInInventory": "2554000000",
            "profitLoss": "4368000000",
            "cashflowFromInvestment": "7375000000",
            "cashflowFromFinancing": "-11617000000",
            "proceedsFromRepaymentsOfShortTermDebt": "None",
            "paymentsForRepurchaseOfCommonStock": "10039000000",
            "paymentsForRepurchaseOfEquity": "10039000000",
            "paymentsForRepurchaseOfPreferre

In [49]:
pretty_print(retrieve_data('EARNINGS', 'NVDA', api_key))

{
    "symbol": "NVDA",
    "annualEarnings": [
        {
            "fiscalDateEnding": "2023-04-30",
            "reportedEPS": "1.09"
        },
        {
            "fiscalDateEnding": "2023-01-31",
            "reportedEPS": "3.33"
        },
        {
            "fiscalDateEnding": "2022-01-31",
            "reportedEPS": "4.45"
        },
        {
            "fiscalDateEnding": "2021-01-31",
            "reportedEPS": "2.51"
        },
        {
            "fiscalDateEnding": "2020-01-31",
            "reportedEPS": "1.45"
        },
        {
            "fiscalDateEnding": "2019-01-31",
            "reportedEPS": "1.66"
        },
        {
            "fiscalDateEnding": "2018-01-31",
            "reportedEPS": "1.15"
        },
        {
            "fiscalDateEnding": "2017-01-31",
            "reportedEPS": "0.64"
        },
        {
            "fiscalDateEnding": "2016-01-31",
            "reportedEPS": "0.3"
        },
        {
            "fiscalDateEnding": "2